# LLMRails

>[LLMRails](https://www.llmrails.com/) 是一个用于构建 GenAI 应用程序的 API 平台。它提供了一个易于使用的 API，用于文档索引和查询，该 API 由 LLMRails 管理，并针对性能和准确性进行了优化。
有关如何使用该 API 的更多信息，请参阅 [LLMRails API 文档](https://docs.llmrails.com/)。

您需要安装 `langchain-community`，指令为 `pip install -qU langchain-community`，以使用此集成。

本笔记展示了如何使用与 Langchain 集成的 `LLMRails` 相关的功能。
请注意，与此类别中的许多其他集成不同，LLMRails 为检索增强生成提供了一个端到端的托管服务，其中包括：
1. 一种从文档文件中提取文本并将其分割成句子的方法。
2. 其 নিজস্ব 嵌入模型和向量存储 - 每个文本片段都被编码成向量嵌入并存储在 LLMRails 内部向量存储中。
3. 一个查询服务，可自动将查询编码为嵌入，并检索最相关的文本片段（包括对 [混合搜索](https://docs.llmrails.com/datastores/search) 的支持）。

所有这些功能都在此 LangChain 集成中得到支持。

# 设置

你需要一个 LLMRails 账户才能在 LangChain 中使用 LLMRails。请按照以下步骤开始：
1. 如果你还没有 LLMRails 账户，请先[注册](https://console.llmrails.com/signup)。
2. 接下来你需要创建 API 密钥来访问 API。在语料库视图中点击 **“API Keys”** 标签页，然后点击 **“Create API Key”** 按钮。为你的密钥命名。点击“Create key”，你现在就拥有了一个激活的 API 密钥。请妥善保管此密钥。

要在 LangChain 中使用 LLMRails，你需要以下值：`api_key`。
你可以通过两种方式将这些值提供给 LangChain：

1. 在你的环境中包含这两个变量：`LLM_RAILS_API_KEY`、`LLM_RAILS_DATASTORE_ID`。

> 例如，你可以使用 `os.environ` 和 `getpass` 来设置这些变量，如下所示：

```python
import os
import getpass

os.environ["LLM_RAILS_API_KEY"] = getpass.getpass("LLMRails API Key:")
os.environ["LLM_RAILS_DATASTORE_ID"] = getpass.getpass("LLMRails Datastore Id:")
```

1. 在创建 LLMRails vectorstore 对象时将它们作为参数提供：

```python
vectorstore = LLMRails(
    api_key=llm_rails_api_key,
    datastore_id=datastore_id
)
```

## 添加文本

要向您的数据存储添加文本，首先需要转到 [Datastores](https://console.llmrails.com/datastores) 页面并创建一个。点击“Create Datastore”按钮，然后为您的数据存储选择一个名称和嵌入模型。接着，从新创建的数据存储设置中获取您的数据存储 ID。

In [19]:
%pip install tika

  Preparing metadata (setup.py) ... done
  Created wheel for tika: filename=tika-2.6.0-py3-none-any.whl size=32621 sha256=b3f03c9dbd7f347d712c49027704d48f1a368f31560be9b4ee131f79a52e176f
  Stored in directory: /Users/omaraly/Library/Caches/pip/wheels/27/ba/2f/37420d1191bdae5e855d69b8e913673045bfd395cbd78ad697
Successfully built tika

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [37]:
import os

from langchain_community.vectorstores import LLMRails

os.environ["LLM_RAILS_DATASTORE_ID"] = "Your datastore id "
os.environ["LLM_RAILS_API_KEY"] = "Your API Key"

llm_rails = LLMRails.from_texts(["Your text here"])

## 相似性搜索

LLMRails 最简单的使用场景是执行相似性搜索。

In [39]:
query = "What do you plan to do about national security?"
found_docs = llm_rails.similarity_search(query, k=5)

In [40]:
print(found_docs[0].page_content)

6  N A T I O N A L S E C U R I T Y S T R A T E G Y Page 7 

This National Security Strategy lays out our plan to achieve a better future of a free, open, secure, and prosperous world.

Our strategy is rooted in our national interests: to protect the security of the American people; to expand economic prosperity and opportunity; and to realize and defend the democratic values at the heart of the American way of life.

We can do none of this alone and we do not have to.

Most nations around the world define their interests in ways that are compatible with ours.

We will build the strongest and broadest possible coalition of nations that seek to cooperate with each other, while competing with those powers that offer a darker vision and thwarting their efforts to threaten our interests.

Our Enduring Role The need for a strong and purposeful American role in the world has never been greater.

The world is becoming more divided and unstable.

Global increases in inflation since the CO

## 相似性搜索及评分

有时我们的目标是进行搜索，同时还需要获取相关性得分，以了解特定结果的好坏程度。

In [41]:
query = "What is your approach to national defense"
found_docs = llm_rails.similarity_search_with_score(
    query,
    k=5,
)

In [42]:
document, score = found_docs[0]
print(document.page_content)
print(f"\nScore: {score}")

But we will do so as the last resort and only when the objectives and mission are clear and achievable, consistent with our values and laws, alongside non-military tools, and the mission is undertaken with the informed consent of the American people.

Our approach to national defense is described in detail in the 2022 National Defense Strategy.

Our starting premise is that a powerful U.S. military helps advance and safeguard vital U.S. national interests by backstopping diplomacy, confronting aggression, deterring conflict, projecting strength, and protecting the American people and their economic interests.

Amid intensifying competition, the military’s role is to maintain and gain warfighting advantages while limiting those of our competitors.

The military will act urgently to sustain and strengthen deterrence, with the PRC as its pacing challenge.

We will make disciplined choices regarding our national defense and focus our attention on the military’s primary responsibilities: to

## LLMRails 作为检索器

LLMRails，如同其他 LangChain 的向量存储一样，通常被用作 LangChain 的检索器：

In [43]:
retriever = llm_rails.as_retriever()
retriever

LLMRailsRetriever(vectorstore=<langchain_community.vectorstores.llm_rails.LLMRails object at 0x1235b0e50>)

In [44]:
query = "What is your approach to national defense"
retriever.invoke(query)

[Document(page_content='But we will do so as the last resort and only when the objectives and mission are clear and achievable, consistent with our values and laws, alongside non-military tools, and the mission is undertaken with the informed consent of the American people.\n\nOur approach to national defense is described in detail in the 2022 National Defense Strategy.\n\nOur starting premise is that a powerful U.S. military helps advance and safeguard vital U.S. national interests by backstopping diplomacy, confronting aggression, deterring conflict, projecting strength, and protecting the American people and their economic interests.\n\nAmid intensifying competition, the military’s role is to maintain and gain warfighting advantages while limiting those of our competitors.\n\nThe military will act urgently to sustain and strengthen deterrence, with the PRC as its pacing challenge.\n\nWe will make disciplined choices regarding our national defense and focus our attention on the milit